In [2]:
import pandas as pd
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import nltk.collocations as collocations
import re
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score, f1_score
import pickle

In [3]:
#f = open('./movie_reviews/train.tsv', 'r')
f = open('train.tsv', 'r')
# loop over lines in the file and use the first limit of them
phrasedata = []
for line in f:
  # ignore the first line starting with Phrase and read all lines
  if (not line.startswith('Phrase')):
    # remove final end of line character
    line = line.strip()
    # each line has 4 items separated by tabs
    # ignore the phrase and sentence ids, and keep the phrase and sentiment
    phrasedata.append(line.split('\t')[2:4])
f.close()

In [4]:
import random

In [5]:
random.shuffle(phrasedata)
phraselist = phrasedata
for phrase in phraselist[:10]:
    print(phrase)

['since 1997', '2']
['that undercuts its charm', '2']
['suggest that the wayward wooden one end it all by stuffing himself into an electric pencil sharpener', '1']
['a great writer and', '3']
['saw this one', '2']
['While the frequent allusions to gurus and doshas will strike some Westerners as verging on mumbo-jumbo ... broad streaks of common sense emerge with unimpeachable clarity .', '3']
["-LRB- Washington 's -RRB-", '2']
['the film is superficial and will probably be of interest primarily to its target audience', '1']
['will the actors generate', '2']
['a heroine who comes across as both shallow and dim-witted', '1']


In [6]:
phrasedocs = []
for phrase in phraselist:
    tokens = nltk.word_tokenize(phrase[0])
    phrasedocs.append((tokens, int(phrase[1])))

In [7]:
for phrase in phrasedocs[:10]:
    print(phrase)

(['since', '1997'], 2)
(['that', 'undercuts', 'its', 'charm'], 2)
(['suggest', 'that', 'the', 'wayward', 'wooden', 'one', 'end', 'it', 'all', 'by', 'stuffing', 'himself', 'into', 'an', 'electric', 'pencil', 'sharpener'], 1)
(['a', 'great', 'writer', 'and'], 3)
(['saw', 'this', 'one'], 2)
(['While', 'the', 'frequent', 'allusions', 'to', 'gurus', 'and', 'doshas', 'will', 'strike', 'some', 'Westerners', 'as', 'verging', 'on', 'mumbo-jumbo', '...', 'broad', 'streaks', 'of', 'common', 'sense', 'emerge', 'with', 'unimpeachable', 'clarity', '.'], 3)
(['-LRB-', 'Washington', "'s", '-RRB-'], 2)
(['the', 'film', 'is', 'superficial', 'and', 'will', 'probably', 'be', 'of', 'interest', 'primarily', 'to', 'its', 'target', 'audience'], 1)
(['will', 'the', 'actors', 'generate'], 2)
(['a', 'heroine', 'who', 'comes', 'across', 'as', 'both', 'shallow', 'and', 'dim-witted'], 1)


In [8]:
docs = []
for phrase in phrasedocs:
    lowerphrase = ([w.lower() for w in phrase[0]], phrase[1])
    docs.append(lowerphrase)
for phrase in docs[:10]:
    print(phrase)

(['since', '1997'], 2)
(['that', 'undercuts', 'its', 'charm'], 2)
(['suggest', 'that', 'the', 'wayward', 'wooden', 'one', 'end', 'it', 'all', 'by', 'stuffing', 'himself', 'into', 'an', 'electric', 'pencil', 'sharpener'], 1)
(['a', 'great', 'writer', 'and'], 3)
(['saw', 'this', 'one'], 2)
(['while', 'the', 'frequent', 'allusions', 'to', 'gurus', 'and', 'doshas', 'will', 'strike', 'some', 'westerners', 'as', 'verging', 'on', 'mumbo-jumbo', '...', 'broad', 'streaks', 'of', 'common', 'sense', 'emerge', 'with', 'unimpeachable', 'clarity', '.'], 3)
(['-lrb-', 'washington', "'s", '-rrb-'], 2)
(['the', 'film', 'is', 'superficial', 'and', 'will', 'probably', 'be', 'of', 'interest', 'primarily', 'to', 'its', 'target', 'audience'], 1)
(['will', 'the', 'actors', 'generate'], 2)
(['a', 'heroine', 'who', 'comes', 'across', 'as', 'both', 'shallow', 'and', 'dim-witted'], 1)


In [9]:
all_words_list = [word for (sent, cat) in docs for word in sent]
all_words = nltk.FreqDist(all_words_list)
print(len(all_words))

16537


In [10]:
word_items = all_words.most_common(1000)
word_features = [word for (word, count) in word_items]

In [11]:
import numpy as np
doc_chunks = np.array_split(np.array(docs), 100)

<ipython-input-11-d1b16b106a7c>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  doc_chunks = np.array_split(np.array(docs), 100)


In [12]:
len(doc_chunks)

100

In [13]:
doc_chunks[0][0][0]

['since', '1997']

In [14]:
docs[0]

(['since', '1997'], 2)

In [15]:
from nltk.collocations import *
import re

#data cleaning and preprocessing
stopwords = nltk.corpus.stopwords.words('english')

def alpha(w):
    pattern = re.compile('^[^a-z]+$')
    if(pattern.match(w)):
        return True
    else:
        return False

#creating bigrams features for the corpus and applying cleaning steps
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(all_words)
finder.apply_word_filter(alpha)
finder.apply_word_filter(lambda w: w in stopwords)
scored = finder.score_ngrams(bigram_measures.raw_freq)
scored[:10]

[(("'30s", 'friendships'), 6.047046018020197e-05),
 (("'40s", 'stuffing'), 6.047046018020197e-05),
 (("'50s", 'frustratingly'), 6.047046018020197e-05),
 (("'60s", 'sensual'), 6.047046018020197e-05),
 (("'70s", 'represents'), 6.047046018020197e-05),
 (("'80s", 'brother-man'), 6.047046018020197e-05),
 (("'90s", 'norm'), 6.047046018020197e-05),
 (("'d", 'intelligence'), 6.047046018020197e-05),
 (("'em", 'ambition'), 6.047046018020197e-05),
 (("'ll", 'history'), 6.047046018020197e-05)]

In [16]:
#extracting clean bigrams (no frequency information)
bigram_features = [bigram for (bigram, count) in scored[:1000]]
#printing the first 30 for confirmation
bigram_features[:30]

[("'30s", 'friendships'),
 ("'40s", 'stuffing'),
 ("'50s", 'frustratingly'),
 ("'60s", 'sensual'),
 ("'70s", 'represents'),
 ("'80s", 'brother-man'),
 ("'90s", 'norm'),
 ("'d", 'intelligence'),
 ("'em", 'ambition'),
 ("'ll", 'history'),
 ("'m", 'emotionally'),
 ("'re", 'feel'),
 ("'til", 'swiftly'),
 ("'ve", 'lot'),
 ('-lrb-', 'good'),
 ('10-course', 'banquet'),
 ('10-year', 'delay'),
 ('10-year-old', 'ontiveros'),
 ('100-minute', 'ok'),
 ('100-year', 'oversimplification'),
 ('102-minute', 'bacon'),
 ('103-minute', 'glimmer'),
 ('10th', 'porthole'),
 ('10th-grade', 'truckzilla'),
 ('112-minute', 'therapeutic'),
 ('12-year-old', 'secretary'),
 ('129-minute', 'rhames'),
 ('12th', 'shaw'),
 ('13th', 'pours'),
 ('14-year-old', 'non-fan')]

In [17]:
def bi_document_features(document, bigram_features):
    document_words = list(nltk.bigrams(document))
    features = {}
    for word in bigram_features:
        #boolean logic will retunt 'True' if there is a match, or 'False' if not
        features['contains({})'.format(word)] = (word in document_words)
    return features

In [18]:
%%time
bigram_chunks = []
for chunk in doc_chunks:
    bigram_chunks.append(
    [(bi_document_features(chunk_sent[0], bigram_features), chunk_sent[1])
    for chunk_sent in chunk]
    )

Wall time: 2min 31s


In [19]:
len(bigram_chunks)

100

In [20]:
import itertools
len(list(itertools.chain.from_iterable(bigram_chunks)))

156060

In [21]:
%%time
featuresets2 = list(itertools.chain.from_iterable(bigram_chunks))

Wall time: 4.01 ms


In [22]:
#checking the length of the featureset
#it should be the same than unigram, because we processed number of documents
#this is for verification
len(featuresets2)

156060

In [26]:
def kfold_naive_bayes(feature_sets, splits=10):
    kf = KFold(n_splits=splits)
    scores = []

    for train, test in kf.split(feature_sets):
        classifier = nltk.NaiveBayesClassifier.train(
            np.array(feature_sets)[train]
        )
        scores.append(
            nltk.classify.accuracy(classifier, 
                                   np.array(feature_sets)[test])
        )
        
    return classifier, scores

In [27]:
def score_model(true_labels, predictions):
    precision = precision_score(true_labels, predictions, average='macro').round(4)
    recall = recall_score(true_labels, predictions, average='macro').round(4)
    f_measure = f1_score(true_labels, predictions, average='macro').round(4)
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F Measure: ", f_measure)
    return precision, recall, f_measure

In [30]:
%%time
uni_10fold_results2 = kfold_naive_bayes(featuresets2, splits = 10)

Wall time: 48min 48s


In [31]:
with open('uni_model_5fold.pickle', 'wb') as f:
#with open("./models/uni_model_5fold.pickle", 'wb') as f:
    pickle.dump(uni_10fold_results2[0], f)

In [32]:
np.mean(uni_10fold_results2[1])

0.5099384851980008

In [33]:
%%time
uni_10fold_pres_labs = [(uni_10fold_results2[0].classify(features), label)
                       for features, label in featuresets2]

Wall time: 14min 9s


In [36]:
preds = []
trues = []
for pred, true in uni_10fold_pres_labs:
    preds.append(pred)
    trues.append(true)

uni_10fold_pres_labs = [trues, preds]
uni_10fold_scores = score_model(uni_10fold_pres_labs[0], uni_10fold_pres_labs[1])

C:\Users\Tom\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Precision:  0.4599
Recall:  0.2002
F Measure:  0.1356
